In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import numpy as np
import pandas as pd
from datasets import load_dataset
from datasets import load_metric
from transformers import pipeline
from pprint import pprint
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import textacy
from textacy import preprocessing
import spacy
import re
import textacy.preprocessing as tp
from datasets import load_dataset, Dataset
import torch
import os
import json
import time
import datetime
import random

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')
model_dump_dir = "./models"
model = GPT2LMHeadModel.from_pretrained(model_dump_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dump_dir)
device = torch.device("cuda")
model.to(device)

2024-04-29 00:33:07.217086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 00:33:08.820410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/tanmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

In [3]:
# model_id = "Narrativaai/BioGPT-Large-finetuned-chatdoctor"

# tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
# device = torch.device('cuda:0')
# model = AutoModelForCausalLM.from_pretrained(model_id)
# model = model.to(device)

# # device = torch.device("cuda")

# def answer_question(
#         prompt,
#         temperature=0.1,
#         top_p=0.75,
#         top_k=40,
#         num_beams=2,
#         **kwargs,
# ):
#     inputs = tokenizer(prompt, return_tensors="pt")
#     input_ids = inputs["input_ids"].to("cuda")
#     attention_mask = inputs["attention_mask"].to("cuda")
#     generation_config = GenerationConfig(
#         temperature=temperature,
#         top_p=top_p,
#         top_k=top_k,
#         num_beams=num_beams,
#         **kwargs,
#     )
#     with torch.no_grad():
#         generation_output = model.generate(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             generation_config=generation_config,
#             return_dict_in_generate=True,
#             output_scores=True,
#             max_new_tokens=512,
#             eos_token_id=tokenizer.eos_token_id

#         )
#     s = generation_output.sequences[0]
#     output = tokenizer.decode(s, skip_special_tokens=True)
#     return output.split(" Response:")[1]

# example_prompt = """
# Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.

# ### Instruction:
# If you are a doctor, please answer the medical questions based on the patient's description.

# ### Input:
# My friend is a gay and had sex with another man. When he woke up this morning he complained about immense pain in his anus and some burning sensation also. Later he found that the person he had sex with is HIV positive. Please advise.

# ### Response:
# """

# print(answer_question(example_prompt))

In [14]:
#test on sample
model.eval()
input_prompt = "I have cancer. Whom should I consult and what medications should I take?"
prompt = f"\n<|startoftext|>[WP] {input_prompt} \n[RESPONSE]"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 800,
                                top_p=0.95, 
                                num_return_sequences=3
                                )
response_string=""
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    response_string+=str(tokenizer.decode(sample_output, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: 
[WP] I have cancer. Whom should I consult and what medications should I take? 
[RESPONSE]Hi, Thanks for writing in. Cancer of the salivary gland is a difficult condition to treat, and you should consult a cancer surgeon immediately if you develop any other symptoms due to this condition. There are many chemotherapy Chat Doctor.  They are given in many cases and can be taken safely. If the cancer is in the initial stage then it is a stage where it is operable and cured completely in a few days. However, you should get your CA 125 done and then take treatment under the guidance of an oncologist. There is no need to take chemotherapy as a regular treatment, and you have to discuss with your oncologist on the above treatments. Please do not worry.


1: 
[WP] I have cancer. Whom should I consult and what medications should I take? 
[RESPONSE]Hello, Thanks for writing to us. If you had cervical lymph node cancer it would have helped us in recommending treatment. The lymph node tuberculos

In [15]:
response_string

"\n[WP] I have cancer. Whom should I consult and what medications should I take? \n[RESPONSE]Hi, Thanks for writing in. Cancer of the salivary gland is a difficult condition to treat, and you should consult a cancer surgeon immediately if you develop any other symptoms due to this condition. There are many chemotherapy Chat Doctor.  They are given in many cases and can be taken safely. If the cancer is in the initial stage then it is a stage where it is operable and cured completely in a few days. However, you should get your CA 125 done and then take treatment under the guidance of an oncologist. There is no need to take chemotherapy as a regular treatment, and you have to discuss with your oncologist on the above treatments. Please do not worry.\n[WP] I have cancer. Whom should I consult and what medications should I take? \n[RESPONSE]Hello, Thanks for writing to us. If you had cervical lymph node cancer it would have helped us in recommending treatment. The lymph node tuberculosis i

In [16]:
nlp = spacy.load("en_core_web_sm", disable=("parser",))
def extract_nouns_verbs(text):
    text = tp.normalize.whitespace(text)
    text = tp.remove.punctuation(text)
    
    doc = nlp(text)

    unique_nouns_verbs = set()
    for token in doc:
        if token.pos_ in ["NOUN","ADJ","ADV"]:
            unique_nouns_verbs.add(token.text)
    
    return " ".join(unique_nouns_verbs)
response_keywords=str(extract_nouns_verbs(response_string))
print(response_keywords)

condition days tumor cases symptoms stage details enough options area lymph completely treatments immediately guidance clinically RESPONSE salivary tuberculosis body many other survival regular weeks patient advanced likely cervical node safely Only treatment However cure surgeon medications then difficult cancer query above oncologist disease need initial concerns Thanks thing same all So prolong chemotherapy care areas operable further poor prognosis gland few WP Hope


In [17]:
df=pd.read_csv("disease_description.csv")

In [18]:
df.head()

,idx,disease,Symptom,reason,TestsAndProcedures,commonMedications
0,0,Panic disorder,"['Anxiety and nervousness', 'Depression', 'Sho...",Panic disorder is an anxiety disorder characte...,"['Psychotherapy', 'Mental health counseling', ...","['Lorazepam', 'Alprazolam (Xanax)', 'Clonazepa..."
1,1,Vocal cord polyp,"['Hoarse voice', 'Sore throat', 'Difficulty sp...","beclomethasone nasal product,","['Tracheoscopy and laryngoscopy with biopsy', ...","['Esomeprazole (Nexium)', 'Beclomethasone Nasa..."
2,2,Turner syndrome,"['Groin mass', 'Leg pain', 'Hip pain', 'Suprap...",Turner syndrome or Ullrich\xe2\x80\x93Turner s...,"['Complete physical skin exam performed (ML)',...","['Somatropin', 'Sulfamethoxazole (Bactrim)', '..."
3,3,Cryptorchidism,"['Symptoms of the scrotum and testes', 'Swelli...",Cryptorchidism (derived from the Greek \xce\xb...,"['Complete physical skin exam performed (ML)',...","['Haemophilus B Conjugate Vaccine (Obsolete)',..."
4,4,Poisoning due to ethylene glycol,"['Abusing alcohol', 'Fainting', 'Hostile behav...","thiamine,","['Intravenous fluid replacement', 'Hematologic...","['Lorazepam', 'Thiamine', 'Naloxone (Suboxone)..."


In [19]:
df.dtypes

idx                    int64
disease               object
Symptom               object
reason                object
TestsAndProcedures    object
commonMedications     object
dtype: object

## Handling score for disease speciality csv

In [20]:
# def handle_diseases_data(document_text):
#     document_text = preprocessing.normalize.whitespace(document_text)
#     document_text = preprocessing.remove.punctuation(document_text)
    
#     en = spacy.load("en_core_web_sm", disable=("parser",))
#     doc = en(document_text)

#     nouns_verbs = [(token.text, token.pos_) for token in doc if token.pos_ in ["NOUN", "ADJ"]]
#     return nouns_verbs

# # Initialize a list to store DataFrames of appended keywords
# appended_keywords_dfs = []

# # Get the maximum number of keywords to fill missing entries
# max_keywords = 0

# # Iterate through each row of the DataFrame
# for index, row in df.iterrows():
#     # Apply the handle_diseases_data function to extract keywords from each column
#     keywords = {}
#     for column in df.columns:
#         if column != 'idx':  # Exclude the 'idx' column
#             extracted_keywords = []
#             if isinstance(row[column], list):
#                 for item in row[column]:
#                     extracted_keywords.extend(handle_diseases_data(item))
#             else:
#                 extracted_keywords.extend(handle_diseases_data(str(row[column])))
            
#             keywords[column] = extracted_keywords
            
#             # Update max_keywords if necessary
#             max_keywords = max(max_keywords, len(extracted_keywords))
    
#     # Convert keywords dictionary to a DataFrame and append it to the list
#     appended_keywords_dfs.append(pd.DataFrame(keywords))

# # Fill missing entries with a placeholder
# for df_keywords in appended_keywords_dfs:
#     for column in df_keywords.columns:
#         current_length = len(df_keywords[column])
#         if current_length < max_keywords:
#             df_keywords[column] += [('', '')] * (max_keywords - current_length)

# # Concatenate all DataFrames in the list along axis 1 (columns)
# appended_keywords_df = pd.concat(appended_keywords_dfs, axis=1)

# # Save the new DataFrame to a CSV file
# appended_keywords_df.to_csv('appending_keywords.csv', index=False)

In [21]:
score_metrics = {
    'disease': 4,
    'Symptom': 2,
    'reason': 1,
    'TestsAndProcedures': 1,
    'commonMedications': 1
}

In [22]:
def calculate_relevance_score(keyword, column_value, column_name):
    if isinstance(column_value, list):
        for item in column_value:
            if keyword in item:
                return score_metrics[column_name]  # Return score metric for the column
    else:
        if keyword in str(column_value):
            return score_metrics[column_name]  # Return score metric for the column
    return 0  # Return 0 if keyword not found in the column

In [23]:
total_scores = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Initialize total score for the current row
    total_score = 0
    
    # Calculate relevance score for each keyword in each column
    for keyword in response_keywords:
        for column_name in df.columns:
            if column_name != 'idx':  # Exclude the 'idx' column
                column_value = row[column_name]
                score = calculate_relevance_score(keyword, column_value, column_name)
                total_score += score
    
    # Append total score for the current row
    total_scores.append(total_score)

# Append total scores to the DataFrame
df['Total_Score'] = total_scores

# Save the updated DataFrame to a CSV file
df.to_csv('updated_dataframe.csv', index=False)

In [24]:
updated_df=pd.read_csv("updated_dataframe.csv")

In [25]:
updated_df.head()

,idx,disease,Symptom,reason,TestsAndProcedures,commonMedications,Total_Score
0,0,Panic disorder,"['Anxiety and nervousness', 'Depression', 'Sho...",Panic disorder is an anxiety disorder characte...,"['Psychotherapy', 'Mental health counseling', ...","['Lorazepam', 'Alprazolam (Xanax)', 'Clonazepa...",3506
1,1,Vocal cord polyp,"['Hoarse voice', 'Sore throat', 'Difficulty sp...","beclomethasone nasal product,","['Tracheoscopy and laryngoscopy with biopsy', ...","['Esomeprazole (Nexium)', 'Beclomethasone Nasa...",3171
2,2,Turner syndrome,"['Groin mass', 'Leg pain', 'Hip pain', 'Suprap...",Turner syndrome or Ullrich\xe2\x80\x93Turner s...,"['Complete physical skin exam performed (ML)',...","['Somatropin', 'Sulfamethoxazole (Bactrim)', '...",3387
3,3,Cryptorchidism,"['Symptoms of the scrotum and testes', 'Swelli...",Cryptorchidism (derived from the Greek \xce\xb...,"['Complete physical skin exam performed (ML)',...","['Haemophilus B Conjugate Vaccine (Obsolete)',...",3119
4,4,Poisoning due to ethylene glycol,"['Abusing alcohol', 'Fainting', 'Hostile behav...","thiamine,","['Intravenous fluid replacement', 'Hematologic...","['Lorazepam', 'Thiamine', 'Naloxone (Suboxone)...",3477


In [26]:
updated_df["Total_Score"].min()

2318

## Doctor To Speciality Mapping

In [27]:
specialist_df = pd.read_csv("specialist_description.csv")
specialist_df.head()

,id,Speciality,Description,Subspeciality
0,1,Allergy and Immunology,Allergy and immunology is a subspecialty of in...,NaN
1,2,Anesthesiology,A doctor of anesthesiologyan anesthesiologistp...,"['adult cardiothoracic anesthesiology', 'criti..."
2,3,Cardiology,Cardiology is a subspecialty of internal medic...,['advanced heart failure and transplant cardio...
3,4,Colon and Rectal Surgery,Colon and rectal surgeons diagnose and treat d...,NaN
4,5,Dermatology,"A doctor of dermatologya dermatologistfinds, p...","['dermatopathology', 'micrographic surgery', '..."


In [36]:
# Initialize a dictionary to store scores for each specialist
scores = {}

# Split the response string into words and convert to lowercase
response_words = response_string.lower().split()

# Iterate over each word in the response string
for word in response_words:
    # Check if the word is in the "Speciality" column of speciality_df
    for index, row in specialist_df.iterrows():
        if word == row['Speciality'].lower() and word != "medicine" and word!="and":
            # Increment score by 1000
            scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 1000
        elif row['Speciality'].lower().endswith("ogy"):
            # Check if the specialty ends with "ogy" and the corresponding term is in the response string
            specialty_term = row['Speciality'].lower()[:-3] + "ogist"
            if specialty_term in response_string.lower():
                scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 1000

# Iterate over each description in speciality_df
for index, row in specialist_df.iterrows():
    description_words = row['Description'].lower().split()
    # Count occurrences of each word in the response string
    for word in description_words:
        if word in response_words:
            # Increase score by 5 for each hit
            scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 5

# Sort specialists based on scores
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Print the top 3 specialists with the highest score
print("Top 3 specialists:")
for i, (speciality, score) in enumerate(sorted_scores[:3], 1):
    print(f"{i}. {speciality}: {score} score")


Top 3 specialists:
1. Oncology: 341055 score
2. Obstetrics and Gynecology: 230 score
3. Allergy and Immunology: 220 score


In [29]:
specialist_df["Speciality"].unique

<bound method Series.unique of 0                   Allergy and Immunology
1                           Anesthesiology
2                               Cardiology
3                 Colon and Rectal Surgery
4                              Dermatology
5                       Emergency Medicine
6                          Family Medicine
7                       Forensic Pathology
8                          General Surgery
9                    Genetics and Genomics
10         Hospice and Palliative Medicine
11                       Hospital Medicine
12                       Internal Medicine
13                               Neurology
14                    Neurological Surgery
15               Obstetrics and Gynecology
16                      Ophthalmic Surgery
17                     Orthopaedic Surgery
18                          Otolaryngology
19                               Pathology
20                              Pediatrics
21    Physical Medicine and Rehabilitation
22                     

## Cosine Similarity

In [30]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer


In [32]:
# Preprocess text data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

# Apply preprocessing to disease descriptions
df['clean_description'] = df['reason'].apply(preprocess_text)
# Apply preprocessing to specialist descriptions
specialist_df['clean_description'] = specialist_df['Description'].apply(preprocess_text)

# Create TF-IDF vectors for disease and specialist descriptions
tfidf_vectorizer = TfidfVectorizer()
disease_tfidf = tfidf_vectorizer.fit_transform(df['clean_description'])
specialist_tfidf = tfidf_vectorizer.transform(specialist_df['clean_description'])

# Calculate cosine similarity between disease and specialist descriptions
similarity_matrix = cosine_similarity(disease_tfidf, specialist_tfidf)

# Get the index of the specialty with the highest similarity score for each disease
max_similarity_index = similarity_matrix.argmax(axis=1)

# Report the specialty name with the highest similarity score for each disease
df['most_similar_speciality'] = specialist_df.iloc[max_similarity_index]['Speciality'].values

# Display the result
print(df[['disease', 'most_similar_speciality']])

                              disease most_similar_speciality
0                      Panic disorder              Pediatrics
1                    Vocal cord polyp  Allergy and Immunology
2                     Turner syndrome   Genetics and Genomics
3                      Cryptorchidism             Dermatology
4    Poisoning due to ethylene glycol  Allergy and Immunology
..                                ...                     ...
791                    Kaposi sarcoma             Dermatology
792                 Spondylolisthesis    Neurological Surgery
793               Pseudotumor cerebri      Ophthalmic Surgery
794       Conjunctivitis due to virus      Ophthalmic Surgery
795            Open wound of the nose  Allergy and Immunology

[796 rows x 2 columns]


### KL-D method

In [33]:
import numpy as np
from scipy.stats import entropy

# Function to calculate KL Divergence
def kl_divergence(p, q):
    return entropy(p, q)

# Calculate KL Divergence between disease and specialist descriptions
kl_divergence_matrix = np.zeros((len(df), len(specialist_df)))
for i, disease_desc in enumerate(df['clean_description']):
    for j, specialist_desc in enumerate(specialist_df['clean_description']):
        disease_words = set(disease_desc.split())
        specialist_words = set(specialist_desc.split())
        
        # Calculate word frequency distributions
        disease_word_freq = {word: disease_desc.count(word) / len(disease_words) for word in disease_words}
        specialist_word_freq = {word: specialist_desc.count(word) / len(specialist_words) for word in disease_words}
        
        # Convert word frequencies to arrays
        p = np.array(list(disease_word_freq.values()))
        q = np.array([specialist_word_freq.get(word, 0) for word in disease_words])
        
        # Add epsilon to avoid divide by zero
        p += np.finfo(float).eps
        q += np.finfo(float).eps
        
        # Calculate KL Divergence
        kl_divergence_matrix[i][j] = kl_divergence(p, q)

# Get the index of the specialty with the lowest KL Divergence for each disease
min_kl_divergence_index = kl_divergence_matrix.argmin(axis=1)

# Report the specialty name with the lowest KL Divergence for each disease
df['most_similar_speciality'] = specialist_df.iloc[min_kl_divergence_index]['Speciality'].values

# Display the result
print(df[['disease', 'most_similar_speciality']])

                              disease    most_similar_speciality
0                      Panic disorder  Obstetrics and Gynecology
1                    Vocal cord polyp     Allergy and Immunology
2                     Turner syndrome            Family Medicine
3                      Cryptorchidism                   Oncology
4    Poisoning due to ethylene glycol     Allergy and Immunology
..                                ...                        ...
791                    Kaposi sarcoma             Anesthesiology
792                 Spondylolisthesis                 Cardiology
793               Pseudotumor cerebri                  Neurology
794       Conjunctivitis due to virus             Anesthesiology
795            Open wound of the nose     Allergy and Immunology

[796 rows x 2 columns]


### Pearson Correlation Coefficient

In [34]:
from scipy.stats import pearsonr

# Calculate Pearson correlation coefficient between disease and specialist descriptions
pearson_corr_matrix = np.zeros((len(df), len(specialist_df)))
for i, disease_desc in enumerate(df['clean_description']):
    for j, specialist_desc in enumerate(specialist_df['clean_description']):
        disease_words = set(disease_desc.split())
        specialist_words = set(specialist_desc.split())
        
        # Create word frequency dictionaries
        disease_word_freq = {word: disease_desc.count(word) for word in disease_words}
        specialist_word_freq = {word: specialist_desc.count(word) for word in disease_words}
        
        # Convert word frequencies to arrays
        p = np.array(list(disease_word_freq.values()))
        q = np.array([specialist_word_freq.get(word, 0) for word in disease_words])
        
        # Pad arrays with zeros to ensure length >= 2
        if len(p) < 2:
            p = np.pad(p, (0, 2 - len(p)), 'constant')
        if len(q) < 2:
            q = np.pad(q, (0, 2 - len(q)), 'constant')
        
        # Calculate Pearson correlation coefficient
        pearson_corr_matrix[i][j], _ = pearsonr(p, q)

# Get the index of the specialty with the highest Pearson correlation coefficient for each disease
max_pearson_corr_index = pearson_corr_matrix.argmax(axis=1)

# Report the specialty name with the highest Pearson correlation coefficient for each disease
df['most_similar_speciality'] = specialist_df.iloc[max_pearson_corr_index]['Speciality'].values

# Display the result
print(df[['disease', 'most_similar_speciality']])

/home/tanmay/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


                              disease most_similar_speciality
0                      Panic disorder      Ophthalmic Surgery
1                    Vocal cord polyp  Allergy and Immunology
2                     Turner syndrome         Family Medicine
3                      Cryptorchidism                Oncology
4    Poisoning due to ethylene glycol  Allergy and Immunology
..                                ...                     ...
791                    Kaposi sarcoma          Anesthesiology
792                 Spondylolisthesis              Cardiology
793               Pseudotumor cerebri              Pediatrics
794       Conjunctivitis due to virus          Anesthesiology
795            Open wound of the nose  Allergy and Immunology

[796 rows x 2 columns]


### Weighted Correlation Co-efficient

In [35]:
def weighted_corrcoef(p, q):
    # Calculate weighted mean of p and q
    weighted_mean_p = np.mean(p)
    weighted_mean_q = np.mean(q)
    
    # Calculate weighted covariance
    weighted_covariance = np.sum((p - weighted_mean_p) * (q - weighted_mean_q)) / len(p)
    
    # Calculate weighted standard deviation of p and q
    weighted_std_p = np.std(p)
    weighted_std_q = np.std(q)
    
    # Calculate weighted correlation coefficient
    weighted_corrcoef = weighted_covariance / (weighted_std_p * weighted_std_q)
    
    return weighted_corrcoef

# Calculate weighted correlation coefficient between disease and specialist descriptions
weighted_corr_matrix = np.zeros((len(df), len(specialist_df)))
for i, disease_desc in enumerate(df['clean_description']):
    for j, specialist_desc in enumerate(specialist_df['clean_description']):
        disease_words = set(disease_desc.split())
        specialist_words = set(specialist_desc.split())
        
        # Create word frequency dictionaries
        disease_word_freq = {word: disease_desc.count(word) for word in disease_words}
        specialist_word_freq = {word: specialist_desc.count(word) for word in disease_words}
        
        # Convert word frequencies to arrays
        p = np.array(list(disease_word_freq.values()))
        q = np.array([specialist_word_freq.get(word, 0) for word in disease_words])
        
        # Calculate weighted correlation coefficient
        weighted_corr_matrix[i][j] = weighted_corrcoef(p, q)

# Get the index of the specialty with the highest weighted correlation coefficient for each disease
max_weighted_corr_index = weighted_corr_matrix.argmax(axis=1)

# Report the specialty name with the highest weighted correlation coefficient for each disease
df['most_similar_speciality'] = specialist_df.iloc[max_weighted_corr_index]['Speciality'].values

# Display the result
print(df[['disease', 'most_similar_speciality']])

/tmp/ipykernel_2840969/2908973671.py:14: RuntimeWarning: invalid value encountered in scalar divide
  weighted_corrcoef = weighted_covariance / (weighted_std_p * weighted_std_q)


                              disease most_similar_speciality
0                      Panic disorder      Ophthalmic Surgery
1                    Vocal cord polyp  Allergy and Immunology
2                     Turner syndrome         Family Medicine
3                      Cryptorchidism                Oncology
4    Poisoning due to ethylene glycol  Allergy and Immunology
..                                ...                     ...
791                    Kaposi sarcoma          Anesthesiology
792                 Spondylolisthesis              Cardiology
793               Pseudotumor cerebri              Pediatrics
794       Conjunctivitis due to virus          Anesthesiology
795            Open wound of the nose  Allergy and Immunology

[796 rows x 2 columns]


In [1]:
from datasets import load_dataset

dataset = load_dataset("medical_dialog", "en")


/home/tanmay/anaconda3/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for medical_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/medical_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


ManualDownloadError:                     The dataset medical_dialog with config en requires manual data.
                    Please follow the manual download instructions:

  For English:
You need to go to https://drive.google.com/drive/folders/1g29ssimdZ6JzTST6Y8g6h-ogUNReBtJD?usp=sharing,    and manually download the dataset from Google Drive. Once it is completed,
    a file named Medical-Dialogue-Dataset-English-<timestamp-info>.zip will appear in your Downloads folder(
    or whichever folder your browser chooses to save files to). Unzip the folder to obtain
    a folder named "Medical-Dialogue-Dataset-English" several text files.

    Now, you can specify the path to this folder for the data_dir argument in the
    datasets.load_dataset(...) option.
    The <path/to/folder> can e.g. be "/Downloads/Medical-Dialogue-Dataset-English".
    The data can then be loaded using the below command:         `datasets.load_dataset("medical_dialog", name="en", data_dir="/Downloads/Medical-Dia

In [ ]:
dataset